In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_validate

In [2]:
df = pd.read_csv("sl ltv.csv")
df.head(10)

,advertising_id,country,install_date,buy_1day,buy_3day,buy_7day,buy_30day,LTV1,LTV3,LTV7,LTV30,session_1day,session_3day,session_7day,session_30day
0,03f5266a-19a5-43ba-a286-a1113b97915e,Brazil,2020-02-13,0,0,0,0,0.0,0.0,0.0,0.0,1,1,1,1
1,056ebb0c-011f-4db9-913b-c6504ff66ad1,Philippines,2020-03-22,0,0,0,0,0.0,0.0,0.0,0.0,1,1,1,1
2,09c0e0e1-d532-4da8-90d2-8208e28a701a,Vietnam,2020-02-01,0,0,0,0,0.0,0.0,0.0,0.0,3,3,3,14
3,16fc24f5-3bf8-466b-9b62-8501118a9a84,Indonesia,2020-02-18,0,0,0,0,0.0,0.0,0.0,0.0,1,5,6,15
4,1ffbb70a-4222-47fc-a4bb-64b180f5f5e5,Iran,2020-04-04,0,0,0,0,0.0,0.0,0.0,0.0,1,5,27,30
5,2eaccdac-1de8-478e-9db6-24de01906de5,Iran,2020-04-11,0,0,0,0,0.0,0.0,0.0,0.0,3,3,4,4
6,30d35732-76d5-457b-864c-bd9cf6827b8c,Vietnam,2020-03-20,0,0,0,0,0.0,0.0,0.0,0.0,3,5,9,15
7,3a0d8a4e-0534-490d-853f-4f506c04c7f6,Philippines,2020-03-20,0,0,0,0,0.0,0.0,0.0,0.0,3,6,7,13
8,40f3a5e5-8e82-4e7f-85c6-64a4fbbfacb6,NaN,2020-03-15,0,0,0,0,0.0,0.0,0.0,0.0,2,3,3,3
9,42601307-3d94-4627-9977-c1ac9552eadb,Japan,2020-03-10,0,0,0,0,0.0,0.0,0.0,0.0,1,1,1,1


In [3]:
df.country.fillna("Unknown", inplace = True)

In [4]:
df["avg_1day"] = df.LTV1 / df.buy_1day
df["avg_3day"] = df.LTV3 / df.buy_3day
df["avg_7day"] = df.LTV7 / df.buy_7day
df["avg_30day"] = df.LTV30 / df.buy_30day
df["avg_1day"].fillna(0, inplace = True)
df["avg_3day"].fillna(0, inplace = True)
df["avg_7day"].fillna(0, inplace = True)
df["avg_30day"].fillna(0, inplace = True)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 872367 entries, 0 to 872366
Data columns (total 19 columns):
advertising_id    872367 non-null object
country           872367 non-null object
install_date      872367 non-null object
buy_1day          872367 non-null int64
buy_3day          872367 non-null int64
buy_7day          872367 non-null int64
buy_30day         872367 non-null int64
LTV1              872367 non-null float64
LTV3              872367 non-null float64
LTV7              872367 non-null float64
LTV30             872367 non-null float64
session_1day      872367 non-null int64
session_3day      872367 non-null int64
session_7day      872367 non-null int64
session_30day     872367 non-null int64
avg_1day          872367 non-null float64
avg_3day          872367 non-null float64
avg_7day          872367 non-null float64
avg_30day         872367 non-null float64
dtypes: float64(8), int64(8), object(3)
memory usage: 126.5+ MB


In [6]:
df.describe()

,buy_1day,buy_3day,buy_7day,buy_30day,LTV1,LTV3,LTV7,LTV30,session_1day,session_3day,session_7day,session_30day,avg_1day,avg_3day,avg_7day,avg_30day
count,872367.000000,872367.000000,872367.000000,872367.000000,872367.000000,872367.000000,872367.000000,872367.000000,872367.000000,872367.000000,872367.000000,872367.000000,872367.000000,872367.000000,872367.000000,872367.000000
mean,0.002129,0.004626,0.006677,0.010171,0.007580,0.015042,0.020787,0.030984,1.707834,3.235785,4.387707,6.128745,0.003400,0.005441,0.006574,0.008229
std,0.088826,0.159802,0.205645,0.320076,0.645962,1.060670,1.242402,1.710492,1.486548,3.146902,5.137749,9.733935,0.151717,0.182231,0.195470,0.227665
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2.000000,2.000000,3.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,4.000000,6.000000,7.000000,0.000000,0.000000,0.000000,0.000000
max,21.000000,42.000000,45.000000,77.000000,353.790000,589.580000,605.550000,795.099824,184.000000,196.000000,212.000000,311.000000,29.663547,29.663547,29.663547,53.985488


In [7]:
df.corr().sort_values("LTV30", ascending = False)

,buy_1day,buy_3day,buy_7day,buy_30day,LTV1,LTV3,LTV7,LTV30,session_1day,session_3day,session_7day,session_30day,avg_1day,avg_3day,avg_7day,avg_30day
LTV30,0.483296,0.606745,0.680660,0.763973,0.601685,0.731361,0.851617,1.000000,0.014587,0.029569,0.044228,0.069832,0.382804,0.452386,0.487950,0.500335
LTV7,0.606161,0.735512,0.750886,0.583121,0.793531,0.943529,1.000000,0.851617,0.015272,0.029166,0.038849,0.041286,0.477987,0.545179,0.560899,0.481981
buy_30day,0.514956,0.685962,0.804739,1.000000,0.354209,0.474738,0.583121,0.763973,0.025926,0.056888,0.082376,0.131068,0.298096,0.376272,0.413018,0.424054
LTV3,0.627078,0.746901,0.662692,0.474738,0.868007,1.000000,0.943529,0.731361,0.015626,0.025445,0.028518,0.025992,0.512198,0.539628,0.500041,0.426689
buy_7day,0.697049,0.910458,1.000000,0.804739,0.500472,0.662692,0.750886,0.680660,0.028434,0.060352,0.080160,0.087911,0.399729,0.489759,0.516044,0.447901
buy_3day,0.784987,1.000000,0.910458,0.685962,0.580033,0.746901,0.735512,0.606745,0.030812,0.055497,0.062021,0.059328,0.453791,0.515671,0.483006,0.414367
LTV1,0.689634,0.580033,0.500472,0.354209,1.000000,0.868007,0.793531,0.601685,0.019412,0.019829,0.019838,0.016983,0.628447,0.499007,0.465153,0.396488
avg_30day,0.372605,0.414367,0.447901,0.424054,0.396488,0.426689,0.481981,0.500335,0.021151,0.046532,0.063937,0.083615,0.638080,0.781962,0.849663,1.000000
avg_7day,0.436586,0.483006,0.516044,0.413018,0.465153,0.500041,0.560899,0.487950,0.023261,0.047031,0.060885,0.062492,0.749802,0.916511,1.000000,0.849663
buy_1day,1.000000,0.784987,0.697049,0.514956,0.689634,0.627078,0.606161,0.483296,0.040694,0.041916,0.041131,0.036236,0.561955,0.464129,0.436586,0.372605


In [11]:
reg = LinearRegression()
scores = cross_validate(reg, df[["LTV7", "LTV3", "buy_7day", "buy_3day"]], df["LTV30"], cv = 5, scoring = ["r2"], return_train_score = True, return_estimator = True)

In [12]:
scores

{'fit_time': array([0.08776498, 0.08676815, 0.08776569, 0.08876228, 0.09773827]),
 'score_time': array([0.00897622, 0.00897646, 0.00997329, 0.00997329, 0.01097107]),
 'estimator': (LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False),
  LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False),
  LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False),
  LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False),
  LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)),
 'test_r2': array([0.7009487 , 0.81210016, 0.84258822, 0.76065991, 0.7377754 ]),
 'train_r2': array([0.82526697, 0.76937192, 0.76322681, 0.77429145, 0.77480582])}

In [18]:
df[df.LTV7 > 0].advertising_id.count() / df.advertising_id.count() * 100

0.26514070339662094

In [19]:
df[df.LTV30 > 0].advertising_id.count() / df.advertising_id.count() * 100

0.33311668139670575

In [13]:
for model in scores["estimator"]:
    print(model.coef_)

[ 1.35214672 -0.37818803  1.19609924 -0.85759468]
[ 2.03920718 -1.12429827 -0.12904057  0.50257683]
[ 2.18401936 -1.28644748 -0.5175493   1.04349367]
[ 2.13892912 -1.22023565 -0.44313322  0.80637489]
[ 2.38226009 -1.43469755 -0.97598338  1.29955548]
